In [1]:
"""
Created on Fri Aug 30 12:34:38 2024
@author: Thierry ALLEM

Prévision des échanges physiques d'électricité en France métropolitaine au cours des années 2020 à 2023 par modélisations XGBoost globale et segmentées
"""

"\nCreated on Fri Aug 30 12:34:38 2024\n@author: Thierry ALLEM\n\nPrévision des échanges physiques d'électricité en France métropolitaine au cours des années 2020 à 2023 par modélisations XGBoost globale et segmentées\n"

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from xgboost import XGBRegressor
import xlsxwriter
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importation du fichier eco2mix_regional_def préparé, incluant les TCO, TCH, cmax_rte et les populations régionales
# Lecture du fichier CSV
df_blackout_ml = pd.read_csv('df_blackout_ML_pop.csv', sep=';',low_memory=False)
# Lecture du fichier CSV
df_blackout_ml.head()

,region,date_heure,annee,jour_numero,heure,jour_fractionnel,thermique,nucleaire,eolien,solaire,...,nucleaire_p_disp,eolien_p_disp,solaire_p_disp,cap_prod_max_hydraulique,ind_prod_hydraulique,hydraulique_p_disp,cap_prod_max_bioenergies,bioenergies_p_disp,region_p_max,population
0,CENTRE VAL DE LOIRE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,90.0,9085.0,508.0,0.0,...,11630.0,662.0,207.0,91.0,0.0,91.0,41.0,41.0,12788.0,2570548
1,PAYS DE LA LOIRE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,127.0,0.0,182.0,0.0,...,0.0,454.0,385.0,5.0,0.0,5.0,57.0,57.0,2738.0,3660852
2,GRAND EST,2013-01-01 00:30:00,2013,1,00:30:00,1.02,319.0,9137.0,1109.0,0.0,...,10820.0,1803.0,498.0,2280.0,0.0,2280.0,79.0,79.0,18878.0,5552388
3,ILE DE FRANCE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,685.0,0.0,16.0,0.0,...,0.0,37.0,108.0,19.0,0.0,19.0,262.0,262.0,2547.0,11959807
4,OCCITANIE,2013-01-01 00:30:00,2013,1,00:30:00,1.02,78.0,2497.0,367.0,0.0,...,2620.0,861.0,1040.0,5368.0,0.0,5368.0,126.0,126.0,10260.0,5683878


In [4]:
# Filtrage des données pour les années 2013 à 2019
df_blackout_ml_13_19= df_blackout_ml[(df_blackout_ml['annee'] >= 2013) & (df_blackout_ml['annee'] <= 2019)].copy()

In [5]:
# Conversion de la colonne 'date_heure' en datetime
df_blackout_ml_13_19['date_heure'] = pd.to_datetime(df_blackout_ml_13_19['date_heure'])

In [6]:
# Conversion de l'heure et des minutes en heure décimale
df_blackout_ml_13_19['heure_decimale'] = df_blackout_ml_13_19['date_heure'].dt.hour + df_blackout_ml_13_19['date_heure'].dt.minute / 60

In [7]:
# ********************************PREVISIONS DES ECHANGES PHYSIQUES PAR MODELISATION XGBoost, méthode globale ********************************

In [8]:
# Encodages cycliques des données temporelles
df_blackout_ml_13_19['heure_decimale_sin'] = np.sin(2 * np.pi * df_blackout_ml_13_19['heure_decimale'] / 24)
df_blackout_ml_13_19['heure_decimale_cos'] = np.cos(2 * np.pi * df_blackout_ml_13_19['heure_decimale'] / 24)

# Fonction pour vérifier si une année est bissextile
def bissextile(annee):
    return annee % 4 == 0 and (annee % 100 != 0 or annee % 400 == 0)

# Ajout d'une colonne pour le nombre total de jours dans l'année en fonction de l'année
df_blackout_ml_13_19['total_jours_annee'] = df_blackout_ml_13_19['annee'].apply(lambda annee: 366 if bissextile(annee) else 365)

# Calcul des encodages cycliques en fonction du nombre de jours dans l'année
df_blackout_ml_13_19['jour_fractionnel_sin'] = np.sin(2 * np.pi * df_blackout_ml_13_19['jour_fractionnel'] / df_blackout_ml_13_19['total_jours_annee'])
df_blackout_ml_13_19['jour_fractionnel_cos'] = np.cos(2 * np.pi * df_blackout_ml_13_19['jour_fractionnel'] / df_blackout_ml_13_19['total_jours_annee'])

# Suppression de la colonne 'total_jours_annee'
df_blackout_ml_13_19= df_blackout_ml_13_19.drop('total_jours_annee', axis=1)

In [9]:
# Sélection des features et de la cible
features = df_blackout_ml_13_19[['region', 'annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                          'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                          'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                          'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                          'cap_prod_max_bioenergies','bioenergies_p_disp',
                          'solaire_p_disp', 'eolien_p_disp', 'population']]

target = df_blackout_ml_13_19['ech_physiques']

In [10]:
# Encodage des variables catégorielles
features = pd.get_dummies(features, columns=['region'])

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Standardisation des données
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Entraînement des modèles avec les hyperparamètres optimaux pour XGBoost
xgb_params = {'subsample': 0.7, 'n_estimators': 400, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.7}

xgb_model_global = XGBRegressor(**xgb_params)

# Entraînement du modèle global
print("Entraînement du modèle XGBoost global...")
xgb_model_global.fit(X_train, y_train)

Entraînement du modèle XGBoost global...


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0.2, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, ...)

In [11]:
# Préparation des données pour les années 2020 à 2023

In [12]:
df_blackout_ml_20_23_glob = df_blackout_ml[(df_blackout_ml['annee'] >= 2020) & (df_blackout_ml['annee'] <= 2023)].copy()

# Conversion de la colonne 'date_heure' en datetime
df_blackout_ml_20_23_glob['date_heure'] = pd.to_datetime(df_blackout_ml_20_23_glob['date_heure'])

# Conversion de l'heure et des minutes en heure décimale
df_blackout_ml_20_23_glob['heure_decimale'] = df_blackout_ml_20_23_glob['date_heure'].dt.hour + df_blackout_ml_20_23_glob['date_heure'].dt.minute / 60

# Encodages cycliques des données temporelles
df_blackout_ml_20_23_glob['heure_decimale_sin'] = np.sin(2 * np.pi * df_blackout_ml_20_23_glob['heure_decimale'] / 24)
df_blackout_ml_20_23_glob['heure_decimale_cos'] = np.cos(2 * np.pi * df_blackout_ml_20_23_glob['heure_decimale'] / 24)

# Ajout d'une colonne pour le nombre total de jours dans l'année en fonction de l'année
df_blackout_ml_20_23_glob['total_jours_annee'] = df_blackout_ml_20_23_glob['annee'].apply(lambda annee: 366 if bissextile(annee) else 365)
    
# Calcul des encodages cycliques en fonction du nombre de jours dans l'année
df_blackout_ml_20_23_glob['jour_fractionnel_sin'] = np.sin(2 * np.pi * df_blackout_ml_20_23_glob['jour_fractionnel'] 
                                                           / df_blackout_ml_20_23_glob['total_jours_annee'])
df_blackout_ml_20_23_glob['jour_fractionnel_cos'] = np.cos(2 * np.pi * df_blackout_ml_20_23_glob['jour_fractionnel'] 
                                                           / df_blackout_ml_20_23_glob['total_jours_annee'])

# Suppression de la colonne 'total_jours_annee'
df_blackout_ml_20_23_glob = df_blackout_ml_20_23_glob.drop('total_jours_annee', axis=1)

# Préparation des features pour les prévisions
features_20_23 = df_blackout_ml_20_23_glob[['region', 'annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                          'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                          'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                          'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                          'cap_prod_max_bioenergies','bioenergies_p_disp',
                          'solaire_p_disp', 'eolien_p_disp', 'population']]

# Encodage des variables catégorielles
features_20_23 = pd.get_dummies(features_20_23, columns=['region'])

# Standardisation des données
features_20_23 = scaler.transform(features_20_23)

# Prévisions avec le modèle global
print("Prévisions avec le modèle XGBoost global...")
df_blackout_ml_20_23_glob['ech_physiques_pred_global'] = xgb_model_global.predict(features_20_23)

# Calcul des métriques globales
y_true = df_blackout_ml_20_23_glob['ech_physiques']
y_pred = df_blackout_ml_20_23_glob['ech_physiques_pred_global']

Prévisions avec le modèle XGBoost global...


In [13]:
# Calcul et exportation des métriques de performance des prévisions

In [14]:
# Calcul de l'erreur absolue
df_blackout_ml_20_23_glob['erreur_absolue_glob'] = np.abs(df_blackout_ml_20_23_glob['ech_physiques_pred_global'] -
                                                          df_blackout_ml_20_23_glob['ech_physiques'])

In [15]:
# Calcul de l'erreur relative seulement pour les valeurs non nulles de 'ech_physiques'

df_blackout_ml_20_23_glob['erreur_relative_glob'] = np.where(
    df_blackout_ml_20_23_glob['ech_physiques'] == 0, np.nan,  # Remplace par NaN lorsque ech_physiques est 0
    (df_blackout_ml_20_23_glob['ech_physiques_pred_global'] - df_blackout_ml_20_23_glob['ech_physiques']) /
    df_blackout_ml_20_23_glob['ech_physiques'] * 100)
metrics_global = {
    'MSE': mean_squared_error(y_true, y_pred),
    'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
    'MAE': mean_absolute_error(y_true, y_pred),
    'R2': r2_score(y_true, y_pred)}

In [16]:
# Création d'un DataFrame pour les métriques globales
df_metrics_global = pd.DataFrame([metrics_global], index=['Global'])

In [17]:
# Calcul des métriques par 'region' sur la période globale 2020 à 2023
metrics_glob_by_region = df_blackout_ml_20_23_glob.groupby(['region']).apply(
    lambda group: pd.Series({
        'MSE': mean_squared_error(group['ech_physiques'], group['ech_physiques_pred_global']),
        'RMSE': np.sqrt(mean_squared_error(group['ech_physiques'], group['ech_physiques_pred_global'])),
        'MAE': mean_absolute_error(group['ech_physiques'], group['ech_physiques_pred_global']),
        'R2': r2_score(group['ech_physiques'], group['ech_physiques_pred_global'])
    })
)

In [18]:
# Calcul des métriques par 'region' et 'annee'
metrics_by_region_year = df_blackout_ml_20_23_glob.groupby(['region', 'annee']).apply(
    lambda group: pd.Series({
        'MSE': mean_squared_error(group['ech_physiques'], group['ech_physiques_pred_global']),
        'RMSE': np.sqrt(mean_squared_error(group['ech_physiques'], group['ech_physiques_pred_global'])),
        'MAE': mean_absolute_error(group['ech_physiques'], group['ech_physiques_pred_global']),
        'R2': r2_score(group['ech_physiques'], group['ech_physiques_pred_global'])
    })
)


In [19]:
# Fusion des métriques globales et par région/année
df_metrics_combined = pd.concat([df_metrics_global, metrics_glob_by_region, metrics_by_region_year])

In [20]:
# Sauvegarde des résultats dans un fichier Excel
with pd.ExcelWriter("xgb_glob_metrics_20_23.xlsx") as writer:
    df_metrics_combined.to_excel(writer, sheet_name='Metriques', float_format='%.4f')

In [21]:
#Suppression des colonnes des variables temporelles encodées
col_to_drop_glob = ['heure_decimale_sin','heure_decimale_cos','jour_fractionnel_sin','jour_fractionnel_cos']

In [22]:
df_blackout_ml_20_23_glob= df_blackout_ml_20_23_glob.drop(col_to_drop_glob, axis = 1)

In [23]:
df_blackout_ml_20_23_glob.head()

,region,date_heure,annee,jour_numero,heure,jour_fractionnel,thermique,nucleaire,eolien,solaire,...,ind_prod_hydraulique,hydraulique_p_disp,cap_prod_max_bioenergies,bioenergies_p_disp,region_p_max,population,heure_decimale,ech_physiques_pred_global,erreur_absolue_glob,erreur_relative_glob
1472244,CENTRE VAL DE LOIRE,2020-01-01,2020,1,00:00:00,1.0,105.0,8013.0,18.0,0.0,...,0.0,91.0,86.0,86.0,13608.0,2574863,0.0,-5515.955078,233.955078,4.429290
1472245,HAUTS DE FRANCE,2020-01-01,2020,1,00:00:00,1.0,2030.0,4507.0,1575.0,0.0,...,0.0,5.0,180.0,180.0,13750.0,5997734,0.0,-1476.450806,174.549194,-10.572332
1472246,PAYS DE LA LOIRE,2020-01-01,2020,1,00:00:00,1.0,359.0,0.0,135.0,0.0,...,0.0,10.0,76.0,76.0,3728.0,3832120,0.0,3392.157715,133.842285,-3.795867
1472247,OCCITANIE,2020-01-01,2020,1,00:00:00,1.0,94.0,2375.0,217.0,0.0,...,128.0,5362.0,146.0,146.0,12293.0,5973969,0.0,664.788391,222.211609,-25.052042
1472248,BOURGOGNE FRANCHE COMTE,2020-01-01,2020,1,00:00:00,1.0,181.0,0.0,247.0,0.0,...,0.0,521.0,69.0,69.0,2199.0,2801695,0.0,1992.910767,144.089233,-6.742594


In [24]:
# Exportation des résultats des prévisions 2020-2023, méthode globale
df_blackout_ml_20_23_glob.to_csv('results_xgb_glob_20_23.csv', index=False)
print("Les prévisions et les métriques ont été calculées et sauvegardées.")

Les prévisions et les métriques ont été calculées et sauvegardées.


In [25]:

#******************************** PREVISIONS DES ECHANGES PHYSIQUES PAR MODELISATION XGBoost, méthode segmentée (régionale) ******************** 

In [26]:
# Initialisation du modèle XGBoost pour les prédictions segmentées
xgb_model_seg = XGBRegressor(**xgb_params)

results_by_region = {}
all_regions_df = []  # Liste pour stocker les DataFrames de chaque région

# Dictionnaire pour stocker les résultats des métriques par région et année
metrics_results_seg = []

# Préparation des données pour les années 2020 à 2023
df_blackout_ml_20_23_seg = df_blackout_ml[(df_blackout_ml['annee'] >= 2020) & (df_blackout_ml['annee'] <= 2023)].copy()

# Conversion de la colonne 'date_heure' en datetime
df_blackout_ml_20_23_seg['date_heure'] = pd.to_datetime(df_blackout_ml_20_23_seg['date_heure'])

# Conversion de l'heure et des minutes en heure décimale
df_blackout_ml_20_23_seg['heure_decimale'] = df_blackout_ml_20_23_seg['date_heure'].dt.hour + df_blackout_ml_20_23_seg['date_heure'].dt.minute / 60

# Encodages cycliques des données temporelles
df_blackout_ml_20_23_seg['heure_decimale_sin'] = np.sin(2 * np.pi * df_blackout_ml_20_23_seg['heure_decimale'] / 24)
df_blackout_ml_20_23_seg['heure_decimale_cos'] = np.cos(2 * np.pi * df_blackout_ml_20_23_seg['heure_decimale'] / 24)

# Ajout d'une colonne pour le nombre total de jours dans l'année en fonction de l'année
df_blackout_ml_20_23_seg['total_jours_annee'] = df_blackout_ml_20_23_seg['annee'].apply(lambda annee: 366 if bissextile(annee) else 365)
    
# Calcul des encodages cycliques en fonction du nombre de jours dans l'année
df_blackout_ml_20_23_seg['jour_fractionnel_sin'] = np.sin(2 * np.pi * df_blackout_ml_20_23_seg['jour_fractionnel'] / df_blackout_ml_20_23_seg['total_jours_annee'])
df_blackout_ml_20_23_seg['jour_fractionnel_cos'] = np.cos(2 * np.pi * df_blackout_ml_20_23_seg['jour_fractionnel'] / df_blackout_ml_20_23_seg['total_jours_annee'])

# Suppression de la colonne 'total_jours_annee'
df_blackout_ml_20_23_seg = df_blackout_ml_20_23_seg.drop('total_jours_annee', axis=1)

for region in df_blackout_ml_13_19['region'].unique():
    print(f"Entraînement du modèle XGBoost pour la région {region}...")
    
    df_region = df_blackout_ml_13_19[df_blackout_ml_13_19['region'] == region].copy()
    features_region_seg = df_region[['annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                              'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                              'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                              'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                              'cap_prod_max_bioenergies','bioenergies_p_disp',
                              'solaire_p_disp', 'eolien_p_disp', 'population']]
    target_region_seg = df_region['ech_physiques']
    
    features_region_seg = pd.get_dummies(features_region_seg)
    
    X_train_region_seg, X_test_region, Y_train_region_seg, y_test_region_seg = train_test_split(features_region_seg, target_region_seg, test_size=0.2, random_state=42)
    
    scaler_region = StandardScaler()
    X_train_region_seg = scaler_region.fit_transform(X_train_region_seg)
    X_test_region = scaler_region.transform(X_test_region)
    
    xgb_model_seg.fit(X_train_region_seg, Y_train_region_seg)
    
    # Préparation des données pour la région (2020-2023)
    df_region_20_23_seg = df_blackout_ml_20_23_seg[df_blackout_ml_20_23_seg['region'] == region].copy()
    features_region_seg_20_23 = df_region_20_23_seg[['annee', 'jour_fractionnel_sin', 'jour_fractionnel_cos', 'heure_decimale_sin', 'heure_decimale_cos',
                              'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                              'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                              'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                              'cap_prod_max_bioenergies','bioenergies_p_disp',
                              'solaire_p_disp', 'eolien_p_disp', 'population']]
    
    features_region_seg_20_23 = pd.get_dummies(features_region_seg_20_23)
    features_region_seg_20_23 = scaler_region.transform(features_region_seg_20_23)
    
    df_region_20_23_seg['ech_physiques_pred_seg'] = xgb_model_seg.predict(features_region_seg_20_23)
   
    # Calcul des métriques de performance par année pour cette région
    for annee in df_region_20_23_seg['annee'].unique():
        df_region_annee_seg = df_region_20_23_seg[df_region_20_23_seg['annee'] == annee]
        
        mse = mean_squared_error(df_region_annee_seg['ech_physiques'], df_region_annee_seg['ech_physiques_pred_seg'])
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(df_region_annee_seg['ech_physiques'], df_region_annee_seg['ech_physiques_pred_seg'])
        r2 = r2_score(df_region_annee_seg['ech_physiques'], df_region_annee_seg['ech_physiques_pred_seg'])
        
        # Stockage des métriques
        metrics_results_seg.append({
            'region': region,
            'annee': annee,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2
        })
            
    # Stockage des résultats par région
    results_by_region[region] = df_region_20_23_seg
    
    # Ajout du DataFrame de cette région à la liste pour concaténation globale
    all_regions_df.append(df_region_20_23_seg)
    
    # Exportation des résultats individuels
    df_region_20_23_seg.to_csv(f"resultats_prev_xgb_{region}.csv", index=False)

# Concaténation de tous les DataFrames en un seul DataFrame global
df_all_regions = pd.concat(all_regions_df, ignore_index=True)

Entraînement du modèle XGBoost pour la région CENTRE VAL DE LOIRE...
Entraînement du modèle XGBoost pour la région PAYS DE LA LOIRE...
Entraînement du modèle XGBoost pour la région GRAND EST...
Entraînement du modèle XGBoost pour la région ILE DE FRANCE...
Entraînement du modèle XGBoost pour la région OCCITANIE...
Entraînement du modèle XGBoost pour la région NORMANDIE...
Entraînement du modèle XGBoost pour la région PROVENCE ALPES COTE D AZUR...
Entraînement du modèle XGBoost pour la région BOURGOGNE FRANCHE COMTE...
Entraînement du modèle XGBoost pour la région HAUTS DE FRANCE...
Entraînement du modèle XGBoost pour la région BRETAGNE...
Entraînement du modèle XGBoost pour la région AUVERGNE RHONE ALPES...
Entraînement du modèle XGBoost pour la région NOUVELLE AQUITAINE...


In [27]:
# Calcul de l'erreur absolue
df_all_regions['erreur_absolue_seg'] = np.abs(df_all_regions['ech_physiques_pred_seg'] - df_all_regions['ech_physiques'])

In [28]:
# Ajout d'une colonne 'erreur relative_seg' en %
df_all_regions['erreur_relative_seg'] = np.where(
    df_all_regions['ech_physiques'] == 0, np.nan,  # Remplace par NaN lorsque ech_physiques est 0
    (df_all_regions['ech_physiques_pred_seg'] - df_all_regions['ech_physiques']) / df_all_regions['ech_physiques'] * 100
)

In [29]:
# Exportation du DataFrame global
df_all_regions.to_csv("resultats_prev_xgb_seg_20_23.csv", index=False)

In [30]:
# Conversion des résultats des métriques en DataFrame
df_metrics = pd.DataFrame(metrics_results_seg)

In [31]:
# Enregistrement des métriques dans un fichier Excel
output_excel = "resultats_xgb_seg_metrics_20_23_year.xlsx"
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    df_metrics.to_excel(writer, index=False, sheet_name='Metriques')

In [32]:
# Calcul des métriques agrégées par région pour l'ensemble des années
aggregated_metrics = df_metrics.groupby('region').agg({
    'MSE': 'mean',
    'RMSE': 'mean',
    'MAE': 'mean',
    'R²': 'mean'
}).reset_index()

In [33]:
# Enregistrement des métriques agrégées dans un nouveau fichier Excel
aggregated_output_excel = "resultats_xgb_seg_metrics_aggregated_20_23.xlsx"
with pd.ExcelWriter(aggregated_output_excel, engine='xlsxwriter') as writer:
    aggregated_metrics.to_excel(writer, index=False, sheet_name='Metriques_Aggreges')

print("Résultats exportés avec succès dans les fichiers Excel.")

Résultats exportés avec succès dans les fichiers Excel.


In [34]:
# Suppression des colonnes des variables temporelles encodées
col_to_drop_seg = ['heure_decimale_sin','heure_decimale_cos','jour_fractionnel_sin','jour_fractionnel_cos']

df_all_regions= df_all_regions.drop(col_to_drop_seg, axis = 1)

df_all_regions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649152 entries, 0 to 649151
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   region                    649152 non-null  object        
 1   date_heure                649152 non-null  datetime64[ns]
 2   annee                     649152 non-null  int64         
 3   jour_numero               649152 non-null  int64         
 4   heure                     649152 non-null  object        
 5   jour_fractionnel          649152 non-null  float64       
 6   thermique                 649152 non-null  float64       
 7   nucleaire                 649152 non-null  float64       
 8   eolien                    649152 non-null  float64       
 9   solaire                   649152 non-null  float64       
 10  hydraulique               649152 non-null  float64       
 11  bioenergies               649152 non-null  float64       
 12  po

In [35]:
df_blackout_ml_20_23_glob.info()

<class 'pandas.core.frame.DataFrame'>
Index: 649152 entries, 1472244 to 2121395
Data columns (total 33 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   region                     649152 non-null  object        
 1   date_heure                 649152 non-null  datetime64[ns]
 2   annee                      649152 non-null  int64         
 3   jour_numero                649152 non-null  int64         
 4   heure                      649152 non-null  object        
 5   jour_fractionnel           649152 non-null  float64       
 6   thermique                  649152 non-null  float64       
 7   nucleaire                  649152 non-null  float64       
 8   eolien                     649152 non-null  float64       
 9   solaire                    649152 non-null  float64       
 10  hydraulique                649152 non-null  float64       
 11  bioenergies                649152 non-null  float6

In [36]:
# Fusion des 2 dataframes

df_blackout_xgb_20_23_pred = (pd.merge(df_blackout_ml_20_23_glob, df_all_regions,
                                       on=["region", "date_heure", 'heure', "annee", "jour_numero",
                                    'jour_fractionnel', 'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies',
                                    'pompage', 'stockage_batterie', 'destockage_batterie', 'cap_prod_max_thermique','thermique_p_disp',
                                     'cap_prod_max_nucleaire', 'nucleaire_p_disp', 'cap_prod_max_hydraulique', 'hydraulique_p_disp',
                                     'cap_prod_max_bioenergies','bioenergies_p_disp', 'eolien_p_disp', 'solaire_p_disp','ind_prod_hydraulique',
                                                                                'region_p_max', 'heure_decimale', 'ech_physiques','population']))
df_blackout_xgb_20_23_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649152 entries, 0 to 649151
Data columns (total 36 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   region                     649152 non-null  object        
 1   date_heure                 649152 non-null  datetime64[ns]
 2   annee                      649152 non-null  int64         
 3   jour_numero                649152 non-null  int64         
 4   heure                      649152 non-null  object        
 5   jour_fractionnel           649152 non-null  float64       
 6   thermique                  649152 non-null  float64       
 7   nucleaire                  649152 non-null  float64       
 8   eolien                     649152 non-null  float64       
 9   solaire                    649152 non-null  float64       
 10  hydraulique                649152 non-null  float64       
 11  bioenergies                649152 non-null  float64 

In [37]:
# Suppression de colonnes
col_to_drop = ['region_p_max','ind_prod_hydraulique']
df_blackout_xgb_20_23_pred= df_blackout_xgb_20_23_pred.drop(col_to_drop, axis = 1)

In [38]:
# Exportation du DataFrame contenant les prévisions globales et régionales
df_blackout_xgb_20_23_pred.to_csv("resultats_prev_xgb_20_23_complet.csv", index=False)

In [39]:
df_blackout_xgb_20_23_pred.head()

,region,date_heure,annee,jour_numero,heure,jour_fractionnel,thermique,nucleaire,eolien,solaire,...,cap_prod_max_bioenergies,bioenergies_p_disp,population,heure_decimale,ech_physiques_pred_global,erreur_absolue_glob,erreur_relative_glob,ech_physiques_pred_seg,erreur_absolue_seg,erreur_relative_seg
0,CENTRE VAL DE LOIRE,2020-01-01,2020,1,00:00:00,1.0,105.0,8013.0,18.0,0.0,...,86.0,86.0,2574863,0.0,-5515.955078,233.955078,4.429290,-5340.710938,58.710938,1.111529
1,HAUTS DE FRANCE,2020-01-01,2020,1,00:00:00,1.0,2030.0,4507.0,1575.0,0.0,...,180.0,180.0,5997734,0.0,-1476.450806,174.549194,-10.572332,-1572.707153,78.292847,-4.742147
2,PAYS DE LA LOIRE,2020-01-01,2020,1,00:00:00,1.0,359.0,0.0,135.0,0.0,...,76.0,76.0,3832120,0.0,3392.157715,133.842285,-3.795867,3441.192627,84.807373,-2.405201
3,OCCITANIE,2020-01-01,2020,1,00:00:00,1.0,94.0,2375.0,217.0,0.0,...,146.0,146.0,5973969,0.0,664.788391,222.211609,-25.052042,693.200989,193.799011,-21.848818
4,BOURGOGNE FRANCHE COMTE,2020-01-01,2020,1,00:00:00,1.0,181.0,0.0,247.0,0.0,...,69.0,69.0,2801695,0.0,1992.910767,144.089233,-6.742594,1969.225952,167.774048,-7.850915


In [40]:
df_blackout_xgb_20_23_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649152 entries, 0 to 649151
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   region                     649152 non-null  object        
 1   date_heure                 649152 non-null  datetime64[ns]
 2   annee                      649152 non-null  int64         
 3   jour_numero                649152 non-null  int64         
 4   heure                      649152 non-null  object        
 5   jour_fractionnel           649152 non-null  float64       
 6   thermique                  649152 non-null  float64       
 7   nucleaire                  649152 non-null  float64       
 8   eolien                     649152 non-null  float64       
 9   solaire                    649152 non-null  float64       
 10  hydraulique                649152 non-null  float64       
 11  bioenergies                649152 non-null  float64 

In [41]:
print("FIN DU CODE")

FIN DU CODE
